In [2]:
# ============================================================================
# SIMPLE CASE SENSITIVITY TEST
# ============================================================================
print("Testing case-insensitive lookup...")

from plotbot.data_classes.data_types import get_data_type_config

# Test the lookup
result = get_data_type_config('mag_rtn_4sa')

if result:
    print(f"✅ SUCCESS! get_data_type_config('mag_rtn_4sa') found the config!")
    print(f"   Mission: {result.get('mission')}")
    print(f"   Data sources: {result.get('data_sources')}")
    print(f"   PySpedas name: {result.get('pyspedas_name')}")
else:
    print(f"❌ FAILED! get_data_type_config('mag_rtn_4sa') returned None")
    print(f"   This means the function exists but isn't working")

Testing case-insensitive lookup...
✅ SUCCESS! get_data_type_config('mag_rtn_4sa') found the config!
   Mission: psp
   Data sources: ['berkeley', 'spdf']
   PySpedas name: None


In [1]:
# ============================================================================
# MAG DATA DOWNLOAD DIAGNOSTIC
# Run this entire cell and share ALL output
# ============================================================================

import sys
import os
import glob
from datetime import datetime

print("="*80)
print("MAG DATA DOWNLOAD DIAGNOSTIC")
print(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

# ============================================================================
# SECTION 1: IMPORT & SETUP
# ============================================================================
print("\n" + "="*80)
print("1. IMPORT & SETUP")
print("="*80)

try:
    import plotbot
    from plotbot import config, mag_rtn_4sa
    from plotbot.data_classes.data_types import data_types, get_local_path
    from plotbot.data_download_pyspedas import download_spdf_data
    import numpy as np
    print("✅ Plotbot imported successfully")
except Exception as e:
    print(f"❌ Failed to import plotbot: {e}")
    import traceback
    traceback.print_exc()
    raise SystemExit("Cannot continue without plotbot")

# Test parameters
trange = ['2020-01-29/18:00:00', '2020-01-29/20:00:00']
data_type = 'mag_rtn_4sa'

print(f"\nTest time range: {trange}")
print(f"Data type: {data_type}")

# ============================================================================
# SECTION 2: MAG CONFIGURATION
# ============================================================================
print("\n" + "="*80)
print("2. MAG CONFIGURATION")
print("="*80)

try:
    mag_config = data_types.get(data_type)
    
    if not mag_config:
        print(f"❌ CRITICAL: No configuration found for '{data_type}' in data_types!")
        print(f"Available data types: {list(data_types.keys())[:10]}...")
    else:
        print(f"✅ MAG configuration found")
        print(f"\nConfiguration details:")
        for key in ['data_level', 'local_path', 'file_pattern_import', 'pyspedas_name', 'pyspedas_datatype']:
            value = mag_config.get(key, 'NOT SET')
            print(f"  {key:25s}: {value}")
        
        # Check data sources
        data_sources = mag_config.get('data_sources', [])
        print(f"\n  data_sources: {data_sources}")
        if 'spdf' in data_sources:
            print(f"    ✅ SPDF download enabled")
        if 'berkeley' in data_sources:
            print(f"    ✅ Berkeley fallback enabled")
            
except Exception as e:
    print(f"❌ Error reading MAG config: {e}")
    import traceback
    traceback.print_exc()

# ============================================================================
# SECTION 3: PATH ANALYSIS
# ============================================================================
print("\n" + "="*80)
print("3. PATH ANALYSIS")
print("="*80)

try:
    print(f"\nPlotbot config:")
    print(f"  config.data_dir:    {config.data_dir}")
    print(f"  Exists:             {os.path.exists(config.data_dir)}")
    
    print(f"\nExpected MAG paths:")
    
    # Get the local path plotbot expects
    local_path = get_local_path(data_type)
    print(f"  get_local_path():   {local_path}")
    print(f"  Exists:             {os.path.exists(local_path)}")
    
    # Create the path if it doesn't exist
    if not os.path.exists(local_path):
        try:
            os.makedirs(local_path, exist_ok=True)
            print(f"  → Created directory: {local_path}")
        except Exception as e:
            print(f"  → ❌ Could not create directory: {e}")
    
    # Check for 2020 subdirectory
    year_2020_path = os.path.join(local_path, '2020')
    print(f"\n  2020 subdirectory:  {year_2020_path}")
    print(f"  Exists:             {os.path.exists(year_2020_path)}")
    
    if os.path.exists(year_2020_path):
        files_2020 = os.listdir(year_2020_path)
        print(f"  Files in directory: {len(files_2020)}")
        if files_2020:
            print(f"  Sample files:")
            for f in sorted(files_2020)[:3]:
                print(f"    - {f}")
    
    # Check PySpedas default path
    print(f"\nPySpedas paths to check:")
    pyspedas_paths = [
        os.path.join(config.data_dir, 'psp_data', 'fields', 'l2', 'mag_rtn_4_per_cycle'),
        './psp_data/fields/l2/mag_rtn_4_per_cycle',  # Relative path PySpedas might use
        os.path.join(os.getcwd(), 'psp_data', 'fields', 'l2', 'mag_rtn_4_per_cycle')
    ]
    
    for path in pyspedas_paths:
        exists = os.path.exists(path)
        print(f"  {path}")
        print(f"    Exists: {exists}")
        if exists:
            try:
                year_dir = os.path.join(path, '2020')
                if os.path.exists(year_dir):
                    files = os.listdir(year_dir)
                    mag_files = [f for f in files if 'mag' in f.lower() and '20200129' in f]
                    print(f"    Files for 2020-01-29: {len(mag_files)}")
                    if mag_files:
                        print(f"      {mag_files[0]}")
            except:
                pass
    
except Exception as e:
    print(f"❌ Error in path analysis: {e}")
    import traceback
    traceback.print_exc()

# ============================================================================
# SECTION 4: PYSPEDAS DIRECT TEST
# ============================================================================
print("\n" + "="*80)
print("4. PYSPEDAS DIRECT DOWNLOAD TEST")
print("="*80)

try:
    import pyspedas
    print(f"✅ PySpedas imported")
    
    # Get PySpedas parameters from config
    pyspedas_name = mag_config.get('pyspedas_name', 'psp')
    pyspedas_datatype = mag_config.get('pyspedas_datatype', 'mag_rtn_4_per_cycle')
    data_level = mag_config.get('data_level', 'l2')
    
    print(f"\nPySpedas parameters:")
    print(f"  Mission (tvarname): {pyspedas_name}")
    print(f"  Datatype:           {pyspedas_datatype}")
    print(f"  Level:              {data_level}")
    print(f"  Time range:         {trange}")
    
    print(f"\nCalling pyspedas.psp.fields()...")
    
    # Try the direct PySpedas call
    result = pyspedas.psp.fields(
        trange=trange,
        datatype=pyspedas_datatype,
        level=data_level,
        no_update=False,
        downloadonly=True,
        notplot=True,
        get_support_data=True
    )
    
    print(f"\nPySpedas result:")
    print(f"  Type:               {type(result)}")
    print(f"  Value:              {result}")
    print(f"  Is truthy:          {bool(result)}")
    
    if result and len(result) > 0:
        print(f"  ✅ SUCCESS - PySpedas returned {len(result)} file(s)")
        for i, fpath in enumerate(result):
            abs_path = os.path.abspath(fpath)
            exists = os.path.exists(fpath)
            print(f"\n  File [{i}]:")
            print(f"    Path:             {fpath}")
            print(f"    Absolute path:    {abs_path}")
            print(f"    Exists:           {exists}")
            if exists:
                size_mb = os.path.getsize(fpath) / (1024*1024)
                print(f"    Size:             {size_mb:.2f} MB")
                
                # Check if this is where plotbot expects it
                expected_path = get_local_path(data_type)
                if expected_path in abs_path:
                    print(f"    ✅ File is in plotbot's expected directory")
                else:
                    print(f"    ⚠️  File NOT in plotbot's expected directory!")
                    print(f"       Expected:      {expected_path}")
                    print(f"       Actually in:   {os.path.dirname(abs_path)}")
    else:
        print(f"  ❌ FAILED - PySpedas returned empty/None")
        print(f"     This means SPDF doesn't have the data")
        print(f"     Berkeley fallback should trigger if config.data_server='dynamic'")
        
except Exception as e:
    print(f"❌ Error during PySpedas download: {e}")
    import traceback
    traceback.print_exc()

# ============================================================================
# SECTION 5: PLOTBOT'S download_spdf_data() TEST
# ============================================================================
print("\n" + "="*80)
print("5. PLOTBOT'S download_spdf_data() WRAPPER TEST")
print("="*80)

try:
    print(f"Calling download_spdf_data(trange, '{data_type}')...")
    
    plotbot_result = download_spdf_data(trange, data_type)
    
    print(f"\nPlotbot wrapper result:")
    print(f"  Type:               {type(plotbot_result)}")
    print(f"  Value:              {plotbot_result}")
    print(f"  Is truthy:          {bool(plotbot_result)}")
    print(f"  Length:             {len(plotbot_result) if plotbot_result else 0}")
    
    if plotbot_result and len(plotbot_result) > 0:
        print(f"  ✅ SUCCESS")
        for fpath in plotbot_result:
            print(f"    - {fpath}")
            print(f"      Exists: {os.path.exists(fpath)}")
    else:
        print(f"  ❌ FAILED - Wrapper returned empty/None")
        
except Exception as e:
    print(f"❌ Error in download_spdf_data: {e}")
    import traceback
    traceback.print_exc()

# ============================================================================
# SECTION 6: FILE SEARCH IN ALL LOCATIONS
# ============================================================================
print("\n" + "="*80)
print("6. COMPREHENSIVE FILE SEARCH")
print("="*80)

try:
    print(f"\nSearching for MAG files with '20200129' in name...")
    
    # Search in multiple locations
    search_roots = [
        config.data_dir,
        os.getcwd(),
        os.path.expanduser('~')
    ]
    
    all_found_files = []
    
    for root in search_roots:
        if not os.path.exists(root):
            continue
        
        print(f"\nSearching in: {root}")
        try:
            # Limit search depth to avoid scanning entire home directory
            pattern = os.path.join(root, '**', '*20200129*.cdf')
            files = glob.glob(pattern, recursive=True)
            
            # Filter for MAG files
            mag_files = [f for f in files if 'mag' in f.lower() and 'rtn' in f.lower()]
            
            if mag_files:
                print(f"  ✅ Found {len(mag_files)} MAG file(s):")
                for f in mag_files[:5]:  # Show first 5
                    print(f"    - {f}")
                    all_found_files.extend(mag_files)
            else:
                print(f"  No MAG files found in this location")
        except Exception as e:
            print(f"  Error searching: {e}")
    
    if all_found_files:
        print(f"\n✅ Total MAG files found: {len(all_found_files)}")
    else:
        print(f"\n❌ NO MAG files found anywhere!")
        
except Exception as e:
    print(f"❌ Error during file search: {e}")
    import traceback
    traceback.print_exc()

# ============================================================================
# SECTION 7: TEST ACTUAL PLOTBOT WORKFLOW
# ============================================================================
print("\n" + "="*80)
print("7. TEST PLOTBOT WORKFLOW")
print("="*80)

try:
    print(f"\nCalling plotbot(trange, mag_rtn_4sa.br, 1)...")
    
    # Disable display to avoid popup
    plotbot.ploptions.display_figure = False
    plotbot.ploptions.return_figure = True
    
    fig = plotbot.plotbot(trange, mag_rtn_4sa.br, 1)
    
    print(f"✅ Plotbot call completed")
    
    # Check if data loaded
    print(f"\nChecking mag_rtn_4sa.br after plotbot call:")
    
    try:
        br_data = np.array(mag_rtn_4sa.br.data)
        br_shape = br_data.shape if hasattr(br_data, 'shape') else 'scalar'
        br_len = len(br_data) if br_shape != 'scalar' else 0
        
        print(f"  br.data shape:      {br_shape}")
        print(f"  br.data length:     {br_len}")
        
        if br_len > 0:
            print(f"  ✅ Data loaded successfully!")
            print(f"  br.data[0]:         {br_data[0]}")
        else:
            print(f"  ❌ Data is EMPTY or scalar!")
            
    except Exception as e:
        print(f"  ❌ Error accessing br.data: {e}")
    
    try:
        br_dt = np.array(mag_rtn_4sa.br.datetime_array)
        dt_shape = br_dt.shape if hasattr(br_dt, 'shape') else 'scalar'
        dt_len = len(br_dt) if dt_shape != 'scalar' else 0
        
        print(f"\n  datetime_array shape: {dt_shape}")
        print(f"  datetime_array length: {dt_len}")
        
        if dt_len > 0:
            print(f"  ✅ Datetime array loaded!")
            print(f"  datetime_array[0]:  {br_dt[0]}")
        else:
            print(f"  ❌ Datetime array is EMPTY or scalar!")
            
    except Exception as e:
        print(f"  ❌ Error accessing datetime_array: {e}")
    
    # Close figure
    import matplotlib.pyplot as plt
    plt.close(fig)
    
except Exception as e:
    print(f"❌ Error in plotbot workflow test: {e}")
    import traceback
    traceback.print_exc()

# ============================================================================
# SECTION 8: DIAGNOSIS SUMMARY
# ============================================================================
print("\n" + "="*80)
print("8. DIAGNOSIS SUMMARY")
print("="*80)

print("\n📋 KEY FINDINGS:")
print("-" * 40)

issues = []
successes = []

# Check configuration
if 'mag_config' in locals() and mag_config:
    successes.append("MAG configuration exists in data_types")
else:
    issues.append("MAG configuration missing from data_types")

# Check PySpedas download
if 'result' in locals() and result and len(result) > 0:
    successes.append(f"PySpedas download succeeded ({len(result)} files)")
else:
    issues.append("PySpedas download failed or returned empty")

# Check plotbot wrapper
if 'plotbot_result' in locals() and plotbot_result and len(plotbot_result) > 0:
    successes.append("Plotbot download wrapper succeeded")
else:
    issues.append("Plotbot download wrapper failed or returned empty")

# Check final data
if 'br_len' in locals() and br_len > 0:
    successes.append(f"MAG data loaded in plotbot ({br_len} points)")
else:
    issues.append("MAG data NOT loaded in plotbot (empty or scalar)")

# Check files exist
if 'all_found_files' in locals() and len(all_found_files) > 0:
    successes.append(f"Found {len(all_found_files)} MAG file(s) on disk")
else:
    issues.append("No MAG files found on disk")

if successes:
    print("\n✅ SUCCESSES:")
    for item in successes:
        print(f"   • {item}")

if issues:
    print("\n❌ ISSUES:")
    for item in issues:
        print(f"   • {item}")

# Diagnosis
print("\n💡 LIKELY CAUSE:")
print("-" * 40)

if not issues:
    print("✅ Everything working correctly!")
elif "PySpedas download failed" in str(issues) and "No MAG files found" in str(issues):
    print("→ PySpedas cannot download MAG data (might not be on SPDF)")
    print("→ Berkeley fallback should trigger but may not be working")
    print("→ Check get_data.py around line 415 for fallback logic")
elif "PySpedas download succeeded" in str(successes) and "No MAG files found" in str(issues):
    print("→ PySpedas claims download succeeded but files don't exist")
    print("→ PATH MISMATCH: Files downloaded to wrong location")
    print("→ Check where PySpedas downloaded vs where plotbot looks")
elif "Found" in str(successes) and "MAG data NOT loaded" in str(issues):
    print("→ Files exist but plotbot can't load them")
    print("→ Check import_data_function() and file pattern matching")
    print("→ Possible file naming or permissions issue")
else:
    print("→ Check detailed output above for specific failure point")

print("\n" + "="*80)
print("DIAGNOSTIC COMPLETE - Share ALL output above")
print("="*80)


MAG DATA DOWNLOAD DIAGNOSTIC
Generated: 2025-10-01 13:54:36

1. IMPORT & SETUP

🕒 Starting import timing session: plotbot_full_initialization
  ✅ matplotlib.pyplot: 0.436s
  ✅ numpy: 0.000s
  🔧 Starting block: core_components
initialized server_access
initialized global_tracker
initialized plot_config
initialized plot_manager
initialized epad class
initialized epad_hr class
initialized proton class
initialized proton_hr class
initialized ham_class
initialized psp_alpha class
initialized psp_qtn class
initialized psp_orbit class
initialized psp_span_vdf class
initialized data_cubby.
CDF classes added to data_cubby type map.
  ✅ Block 'core_components' completed in 0.841s
  🔧 Starting block: psp_data_classes
initialized proton_fits class
initialized alpha_fits class
  ✅ Block 'psp_data_classes' completed in 0.001s
  🔧 Starting block: wind_data_classes
  ✅ Block 'wind_data_classes' completed in 0.000s
  🔧 Starting block: data_cubby_registration
  ✅ Block 'data_cubby_registration' complete

Traceback (most recent call last):
  File "/var/folders/3n/8nbttjbs573270nf5nvjg3cw0000gn/T/ipykernel_12530/1519282384.py", line 157, in <module>
    pyspedas_name = mag_config.get('pyspedas_name', 'psp')
                    ^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'get'


KeyboardInterrupt: 